In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import pickle

In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
data_path='./data/bind_mark.txt'        #mark 資料

In [3]:
# 輸入path給data list
def Dataset(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data=f.readlines()#.encode('utf-8').decode('utf-8-sig')
    data_list, data_list_tmp = list(), list()
    article_id_list=list()
    idx=0
    for row in data:
        data_tuple = tuple()
        if row == '\n':
            article_id_list.append(idx)
            idx+=1
            data_list.append(data_list_tmp)
            data_list_tmp = []
        else:
            row = row.strip('\n').split(' ')
            data_tuple = (row[0], row[1])
            data_list_tmp.append(data_tuple)
    if len(data_list_tmp) != 0:
        data_list.append(data_list_tmp)
    
    return data_list, article_id_list

In [4]:
# 輸入mark的 data_list 資料給出 Y 值
def Preprocess(data_list):
    label_list = list()
    for idx_list in range(len(data_list)):
        label_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            label_list_tmp.append(data_list[idx_list][idx_tuple][1])
        label_list.append(label_list_tmp)
    return label_list

In [5]:
#輸入為 一個 pickle 檔               用 m.shape[0] 決定 embedding長度  第 8 行
def get_X(data):
    output = []
    for sen in data:
        for m in sen:
            for col in range(1, m.shape[1]-1):
                d = {}
                for row in range(600):
                    d['dim_{}'.format(str(row+1))] = m[row, col]
                output.append(d)
    return output
    

In [6]:
# 獲得 y
data_list, article_id_list = Dataset(data_path)
y = Preprocess(data_list)
len(y)

146

In [8]:
x_train = []
for i in range(1, 147):
    with open('./embedding_data/data{}.pickle'.format(str(i)), 'rb') as f:
        p = pickle.load(f)
    x_train.append(get_X(p))
len(x_train)

146

In [7]:
# x_train[0][0]

NameError: name 'x_train' is not defined

In [12]:
x_test = []
for i in range(1, 41):
    with open('./embedding_data/dev/dev{}.pickle'.format(str(i)), 'rb') as f:
        p = pickle.load(f)
    x_test.append(get_X(p))
len(x_test)

40

In [15]:
# with open('./embedding_data/data{}.pickle'.format(str(3)), 'rb') as f:
#     p = pickle.load(f)

In [16]:
# c = 0
# for sen in p:
#     for m in sen:
#         c += m.shape[1] - 2
# c

1827

In [19]:
# vv = get_X(p)

In [20]:
# len(vv)

1827

In [18]:
# len(y[2])

1827

In [36]:
# x_test = []
# for i in range(1, 41):
#     with open('./embedding_data/dev/dev{}.pickle'.format(str(i)), 'rb') as f:
#         p = pickle.load(f)
#     x_test.append(get_X(p))

In [20]:
len(x_test)

40

In [9]:
def CRF(X_train, y_train):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    
    return crf

In [10]:
crf = CRF(x_train, y)

In [11]:
joblib.dump(crf, './data/model_crf_bert.joblib')

['./data/model_crf_bert.joblib']

In [13]:
y_pred = crf.predict(x_test)

In [44]:
B = 0
for i in range(len(y_pred)):
    for w in range(len(y_pred[i])):
        if y_pred[i][w][0] == 'B':
            B += 1
        
        if y_pred[i][w] != 'O':
            print(i, w, y_pred[i][w])
print(B)

0 1633 B-family
0 1634 I-family
0 1635 I-family
0 1636 B-money
0 1637 I-money
0 1638 I-money
0 1639 B-med_exam
0 1640 I-med_exam
0 1641 I-med_exam
0 1642 I-med_exam
0 1643 I-med_exam
0 1644 I-med_exam
0 1645 I-med_exam
0 1646 I-med_exam
0 1647 I-med_exam
0 1648 I-med_exam
0 2691 B-family
0 2692 I-family
3 1068 B-family
3 1069 I-family
3 1070 I-family
3 1071 I-family
6 994 B-med_exam
6 995 I-med_exam
6 996 I-med_exam
10 262 B-time
10 263 I-time
10 264 I-time
10 265 I-time
12 628 B-name
12 629 I-name
13 60 B-med_exam
13 61 I-med_exam
13 62 I-med_exam
13 166 B-med_exam
13 167 I-med_exam
13 168 I-med_exam
15 1462 B-med_exam
15 1463 I-med_exam
15 1464 I-med_exam
16 324 B-time
16 325 I-time
16 326 I-time
16 327 I-time
16 397 B-med_exam
16 398 I-med_exam
16 399 I-med_exam
16 400 I-med_exam
16 401 I-med_exam
16 402 I-med_exam
16 403 I-med_exam
16 404 I-med_exam
16 405 I-med_exam
21 518 B-organization
21 519 I-organization
21 520 I-organization
21 521 I-organization
26 1447 B-organization
26 14

In [25]:
with open('./data/dev.pickle', 'rb') as f:
    test = pickle.load(f)

In [26]:
testdata = []
for i, j in test.items():
    s = ''
    for k in range(len(j[0])):
        s = s + j[0][k] + j[1][k]
    testdata.append(s)
    

In [27]:
test_id_list = [i for i in range(40)]

In [28]:
len(y_pred)

40

In [45]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for test_id in range(len(y_pred)):
    pos=0
    start_pos=None
    end_pos=None
    entity_text=None
    entity_type=None
    for pred_id in range(len(y_pred[test_id])):
        if pred_id+1 == len(y_pred[test_id]):
            c = 'O'
        else:
            c = y_pred[test_id][pred_id+1][0]
        
        if y_pred[test_id][pred_id][0]=='B':
            start_pos=pos
            entity_type=y_pred[test_id][pred_id][2:]
            
#         elif start
            
        elif start_pos is not None and y_pred[test_id][pred_id][0]=='I' and y_pred[test_id][pred_id+1][0]!='I':
            end_pos=pos
            entity_text=''.join([testdata[test_id][position][0] for position in range(start_pos,end_pos+1)])
            line=str(test_id_list[test_id])+'\t'+str(start_pos)+'\t'+str(end_pos+1)+'\t'+entity_text+'\t'+entity_type
            output+=line+'\n'
            start_pos = None
        pos+=1     

In [48]:
output_path='./data/output11.tsv'
with open(output_path,'w',encoding='utf-8') as f:
    f.write(output)

In [30]:
# y_pred[20][3013] = 'O'

In [46]:
print(output)

article_id	start_position	end_position	entity_text	entity_type
0	1633	1636	眾：啊	family
0	1636	1639	第二次	money
0	1639	1649	是一直拉肚子拉到天亮	med_exam
0	2691	2693	改善	family
3	1068	1072	，為什麼	family
6	994	997	418	med_exam
10	262	266	四個禮拜	time
12	628	630	亞尼	name
13	60	63	9．7	med_exam
13	166	169	9．8	med_exam
15	1462	1465	90分	med_exam
16	324	328	5.02	time
16	397	406	7.83，7.02	med_exam
21	518	522	交友軟體	organization
26	1447	1451	交友軟體	organization
35	4831	4842	1000到1500台幣	money
37	981	986	400多塊	money



In [42]:
testdata[0][1633:1639]

'眾：啊第二次'

In [53]:
len(y_pred)

40